In [ ]:
import pandas as pd


source = "TSLA_1min_firstratedata.csv"
df = pd.read_csv("data/" + source)

df["timestamp"] = pd.to_datetime(df["timestamp"])
df["date"] = df["timestamp"].dt.date
df["time"] = df["timestamp"].dt.time

# Writing the DataFrame to a new CSV file
df.to_csv("data/cleaned_tsla.csv", index=False)

In [ ]:
def RSI(df, n):
    "function to calculate RSI"
    delta = df["close"].diff()
    delta = delta[1:]
    up, down = delta.copy(), delta.copy()
    up[up < 0] = 0
    down[down > 0] = 0
    df["up"] = up.round(4)
    df["down"] = down.round(4)
    AVG_Gain = df["up"].rolling(window=n).mean()
    AVG_Loss = abs(df["down"].rolling(window=n).mean())
    RS = AVG_Gain / AVG_Loss
    RSI = 100.0 - (100.0 / (1.0 + RS))
    df["RSI_14"] = RSI.round(4)
    return df


df = RSI(df, 14)

df.head()

In [ ]:
# Candlestick chart of one day
import datetime
import plotly.graph_objects as go


def chart_range(df, date):
    df = df[df["date"] == date]
    fig = go.Figure(
        data=[
            go.Candlestick(
                x=df["timestamp"],
                open=df["open"],
                high=df["high"],
                low=df["low"],
                close=df["close"],
            )
        ]
    )
    fig.show()


chart_range(df, datetime.date(2023, 5, 18))

In [ ]:
# chart the RSI of one day
def chart_rsi(df, date):
    df = df[df["date"] == date]
    # only chart from 9:30 to 16:00
    df = df[df["time"] >= datetime.time(9, 30, 0)]
    df = df[df["time"] <= datetime.time(16, 0, 0)]
    fig = go.Figure(data=go.Scatter(x=df["timestamp"], y=df["RSI_14"]))
    # add horizontal line at 30 and 70
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=30,
        x1=df["timestamp"].max(),
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=70,
        x1=df["timestamp"].max(),
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.show()


chart_rsi(df, datetime.date(2023, 5, 18))


def chart_rsi_with_trades(timestamps, rsi_values, enters, exits):
    fig = go.Figure(
        data=go.Scatter(x=timestamps, y=rsi_values, mode="lines", name="RSI")
    )
    fig.add_trace(
        go.Scatter(
            x=enters["timestamps"],
            y=enters["values"],
            mode="markers",
            marker=dict(color="green", size=8),
            name="Enter",
        )
    )
    fig.add_trace(
        go.Scatter(
            x=exits["timestamps"],
            y=exits["values"],
            mode="markers",
            marker=dict(color="red", size=8),
            name="Exit",
        )
    )
    # Add horizontal lines at RSI 30 and 70
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=30,
        x1=timestamps[-1],
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=timestamps[0],
        y0=70,
        x1=timestamps[-1],
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )

    fig.show()

In [ ]:
import datetime
import pandas as pd


def simulate_trade_RSI(date, entry, exit, df, debug=False):
    # Filter data for the specified date and time range
    df_day = df[
        (df["date"] == date)
        & (df["time"] >= datetime.time(9, 30, 0))
        & (df["time"] <= datetime.time(16, 0, 0))
    ]

    # Ensure the dataframe is not empty
    if len(df_day) == 0:
        return

    # Initialize variables
    trades = 0
    in_trade = False

    # Create an empty DataFrame for trade log
    trade_log = []

    # Iterate through the dataframe
    for i in range(1, len(df_day)):
        if (
            not in_trade
            and df_day["RSI_14"].iloc[i - 1] < entry
            and df_day["RSI_14"].iloc[i] >= entry
        ):
            # Buy at entry condition
            entry_price = df_day["close"].iloc[i]
            in_trade = True
            timeEnter = df_day["time"].iloc[i - 1]
        elif (
            in_trade
            and df_day["RSI_14"].iloc[i - 1] > exit
            and df_day["RSI_14"].iloc[i] <= exit
        ):
            # Sell at exit condition
            exit_price = df_day["close"].iloc[i]

            # Calculate gain for the trade
            if entry_price != 0:
                trade_gain_percent = ((exit_price - entry_price) / entry_price) * 100
                trade_gain_percent = round(trade_gain_percent, 2)

                trade_gain_net = exit_price - entry_price
                trade_gain_net = round(trade_gain_net, 2)

                trades += 1

                # Append trade details to the trade log DataFrame
                trade_log.append(
                    {
                        "Direction": "Long",
                        "Entry": entry_price,
                        "Exit": exit_price,
                        "TimeEnter": timeEnter,
                        "TimeExit": df_day["time"].iloc[i],
                        "Gain%": trade_gain_percent,
                        "Gain$": trade_gain_net,
                    }
                )

                # Reset trade variables
                entry_price = 0
                exit_price = 0
                in_trade = False

    if debug:
        for e in trade_log:
            print(e)

    output = {
        "RSI Bounds": (entry, exit),
        "Trades": trades,
        "Gain%": sum([e["Gain%"] for e in trade_log]).round(2),
        "Gain$": sum([e["Gain$"] for e in trade_log]).round(2),
    }
    return output


# chart the RSI of one day
def chart_rsi(df, date):
    df = df[df["date"] == date]
    # only chart from 9:30 to 16:00
    df = df[df["time"] >= datetime.time(9, 30, 0)]
    df = df[df["time"] <= datetime.time(16, 0, 0)]
    fig = go.Figure(data=go.Scatter(x=df["timestamp"], y=df["RSI_14"]))
    # add horizontal line at 30 and 70
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=30,
        x1=df["timestamp"].max(),
        y1=30,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.add_shape(
        type="line",
        x0=df["timestamp"].min(),
        y0=70,
        x1=df["timestamp"].max(),
        y1=70,
        line=dict(color="RoyalBlue", width=1, dash="dot"),
    )
    fig.show()


# for e in range(30, 70, 5):
#     simulate_trade_RSI(datetime.date(2023, 5, 18), e, 70, df)


def simulate_date_range(start_date, end_date, entry, exit, df):
    # Iterate through the date range
    sum_trades = 0
    product_gain_percent = 1
    sum_gain_net = 0

    for n in range((end_date - start_date).days + 1):
        date = start_date + datetime.timedelta(n)
        day_data = simulate_trade_RSI(date, entry, exit, df)
        if day_data is not None:
            print(date, day_data)
            sum_trades += day_data["Trades"]
            product_gain_percent *= 1 + (day_data["Gain%"])
            sum_gain_net += day_data["Gain$"]

    status = {
        "Total Trades": sum_trades,
        "Product Gain%": round((product_gain_percent - 1) * 100, 2),
        "Total Gain$": sum_gain_net.round(2),
    }
    print(status)


# simulate_date_range(datetime.date(2023, 5, 1), datetime.date(2023, 5, 31), 30, 50, df)


simulate_trade_RSI(datetime.date(2023, 5, 1), 30, 50, df, debug=True)

In [ ]:
df.head()